In [ ]:
import cv2
import numpy as np

thresh = 25     # 프레임 간 차이 이미지를 이진화할 때 사용할 임계값.
max_diff = 5    # 모션 감지를 판단할 때 사용할 최대 차이 픽셀 수.

# YOLO 모델 설정
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# COCO 클래스 로드
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

a, b, c = None, None, None
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 720)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 600)
if cap.isOpened():
    # 첫, 두 프레임을 읽어옴
    ret, a = cap.read()
    ret, b = cap.read()

    while ret:
        # 세 번째 프레임을 읽어옴
        ret, c = cap.read()
        # 현재 프레임을 복사하여 그리기 작업에 사용
        draw = c.copy()
        if not ret:
            break
        
        # 세 프레임을 그레이스케일로 변환
        a_gray = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
        b_gray = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)
        c_gray = cv2.cvtColor(c, cv2.COLOR_BGR2GRAY)
        
        # 연속된 두 프레임 간의 차이 계산
        diff1 = cv2.absdiff(a_gray, b_gray)
        diff2 = cv2.absdiff(b_gray, c_gray)

        # 차이 이미지에 임계값 적용하여 이진 이미지로 변환
        ret, diff1_t = cv2.threshold(diff1, thresh, 255, cv2.THRESH_BINARY)
        ret, diff2_t = cv2.threshold(diff2, thresh, 255, cv2.THRESH_BINARY)

        # 두 차이 이미지의 비트 AND 연산
        diff = cv2.bitwise_and(diff1_t, diff2_t)

        # 노이즈 제거를 위해 형태학적 연산 적용
        k = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
        diff = cv2.morphologyEx(diff, cv2.MORPH_OPEN, k)

        # 차이 이미지에서 흰색 픽셀의 개수 계산
        diff_cnt = cv2.countNonZero(diff)

        # 차이가 있는 영역이 설정된 최대값보다 크면
        if diff_cnt > max_diff:
            # 차이가 있는 영역의 좌표 계산
            nzero = np.nonzero(diff)
            if len(nzero[0]) > 0 and len(nzero[1]) > 0:  # nzero가 비어있지 않은지 확인
                # 차이 영역에 사각형 그리기
                x1, y1, x2, y2 = min(nzero[1]), min(nzero[0]), max(nzero[1]), max(nzero[0])
                cv2.rectangle(draw, (x1, y1), (x2, y2), (0,255,0), 2)

                # 객체 검출을 위한 블롭 생성
                blob = cv2.dnn.blobFromImage(c, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
                net.setInput(blob)
                outs = net.forward(output_layers)

                # 검출된 객체의 정보 저장
                class_ids = []
                confidences = []
                boxes = []

                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.5:
                            center_x = int(detection[0] * 720)
                            center_y = int(detection[1] * 600)
                            w = int(detection[2] * 720)
                            h = int(detection[3] * 600)
                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

                detected = False
                for i in range(len(boxes)):
                    if i in indexes:
                        x, y, w, h = boxes[i]
                        label = str(classes[class_ids[i]])
                        if label in ["person", "cat", "dog"]:  # 사람, 고양이, 개만 감지
                            color = (0, 255, 0) if label == "person" else (0, 0, 255)
                            # cv2.rectangle(draw, (x, y), (x + w, y + h), color, 2)
                            cv2.putText(draw, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                            detected = True

                if not detected:
                    # 객체가 검출되지 않은 경우에도 모션 감지 영역에 텍스트 추가
                    cv2.putText(draw, 'Motion Detection', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # 현재 프레임과 차이 이미지를 나란히 붙여서 출력
        stacked = np.hstack((draw, cv2.cvtColor(diff, cv2.COLOR_GRAY2BGR)))
        cv2.imshow('motion_sensor', stacked)

        # 다음 프레임을 위해 변수 업데이트
        a = b
        b = c

        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()